In [23]:
# only when open in colab
!git init .
!git remote add origin https://github.com/coyote-bergstein/compression-using-neural-networks.git
!git pull origin speech-commands

Reinitialized existing Git repository in /content/.git/
error: remote origin already exists.
From https://github.com/coyote-bergstein/compression-using-neural-networks
 * branch            speech-commands -> FETCH_HEAD
Already up to date.


In [24]:
!pip install pydub

In [37]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import librosa

In [26]:
# only when open in colab
import importlib
import source
from source.utils import generate_spectrogram, reconstruct_audio
from source.constants import HOP_LENGTH, N_FFT, AUDIO_SAMPLE_RATE
importlib.reload(source.constants)
importlib.reload(source.utils)

<module 'source.utils' from '/content/source/utils.py'>

## Dataset

In [54]:
ds = tfds.load('speech_commands', split='train')


## Audio processing testing

In [56]:
import IPython.display as ipd
from source.utils import generate_spectrogram, reconstruct_audio
from source.constants import HOP_LENGTH, N_FFT, AUDIO_SAMPLE_RATE


signals = []

for example in ds.take(5):
  audio_np = example['audio'].numpy()
  audio_np = audio_np/audio_np.max()
  signals.append(audio_np)


for signal in signals:
  S = generate_spectrogram(signal, HOP_LENGTH, N_FFT)
  print(S.shape)
  reconstructed_signal = reconstruct_audio(S, HOP_LENGTH, N_FFT)

  print("Original Audio:")
  ipd.display(ipd.Audio(signal, rate=AUDIO_SAMPLE_RATE))

  print("Reconstructed Audio:")
  ipd.display(ipd.Audio(reconstructed_signal, rate=AUDIO_SAMPLE_RATE))

print(N_FFT, HOP_LENGTH)

(128, 126)
Original Audio:


Reconstructed Audio:


(128, 126)
Original Audio:


Reconstructed Audio:


(128, 126)
Original Audio:


...

(128, 126)
Original Audio:


Reconstructed Audio:


(128, 126)
Original Audio:


Reconstructed Audio:


512 128


## Define model architecture
### TODO model architecture must be changed

In [57]:
n_mels = 128

In [78]:
def preprocess(example):
    audio = example.numpy()
    audio = audio/audio.max()

    S = generate_spectrogram(audio, HOP_LENGTH, N_FFT)
    return S.astype(np.float32)

def preprocess_with_py_function(example):
    audio = tf.py_function(preprocess, [example['audio']], [tf.float32])
    # audio.set_shape([n_mels, None])
    return audio

In [79]:
ds_train = ds.map(preprocess_with_py_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache().batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [84]:
from tensorflow.keras import layers, models

input_shape = (n_mels, None, 1)

# Encoder
encoder_input = layers.Input(shape=input_shape)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)

# Decoder
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)



In [86]:
from source.constants import OPTIMIZER, LOSS

autoencoder = tf.keras.Model(encoder_input, decoded)
autoencoder.compile(optimizer=OPTIMIZER, loss=LOSS)
autoencoder.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, None, 1)]    0         
                                                                 
 conv2d_47 (Conv2D)          (None, 128, None, 16)     160       
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 64, None, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_48 (Conv2D)          (None, 64, None, 8)       1160      
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 32, None, 8)       0         
 ng2D)                                                           
                                                                 
 conv2d_49 (Conv2D)          (None, 32, None, 8)       584 

In [88]:
history = autoencoder.fit(ds_train, epochs=20, verbose=1)

ValueError: `y` argument is not supported when using dataset as input.